### read data

In [11]:
import pandas as pd
target=pd.read_excel("./第五批.xlsx",sheet_name="Sheet2",skiprows=1)
# target.fillna(0, inplace=True)
# target[['借支金额']].fillna(0, inplace=True)
# target[['还款金额','借支金额']]=target[['还款金额','借支金额']].fillna(0, inplace=True)
target.info()
# target.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577 entries, 0 to 576
Data columns (total 35 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   577 non-null    int64  
 1   Unnamed: 1   67 non-null     object 
 2   批次           577 non-null    object 
 3   患者身份         145 non-null    object 
 4   姓名           577 non-null    object 
 5   性别           576 non-null    object 
 6   年龄           569 non-null    object 
 7   住址           548 non-null    object 
 8   电话           525 non-null    object 
 9   诊断           545 non-null    object 
 10  科室           576 non-null    object 
 11  住院号          577 non-null    int64  
 12  入院时间         577 non-null    object 
 13  出院时间         572 non-null    object 
 14  押金           92 non-null     object 
 15  通道           99 non-null     object 
 16  总额           568 non-null    float64
 17  保障           563 non-null    float64
 18  保险           504 non-null    float64
 19  民政      

In [ ]:
> 处理时间字段：入院时间 出院时间

In [ ]:
查看nan的时间空字段：

In [ ]:
# target[target['入院时间'].isna()] 
target[target['出院时间'].isna()] 

附加20，形成格式2020.01.20

In [12]:
target['入院时间']="20"+(target['入院时间'].str.strip())
target['出院时间']="20"+(target['出院时间'].str.strip())

In [13]:
target['入院时间']

0      2019.10.28
1      2019.11.01
2      2019.11.04
3      2019.11.04
4      2019.11.08
          ...    
572    2018.06.22
573    2018.08.31
574    2018.12.18
575    2019.01.23
576    2018.09.21
Name: 入院时间, Length: 577, dtype: object

In [16]:
test=target[:10]
test['入院时间1']=pd.to_datetime(test['入院时间'],format='%Y.%m.%d',errors='coerce')
test['入院时间1']

0   2019-10-28
1   2019-11-01
2   2019-11-04
3   2019-11-04
4   2019-11-08
5   2019-11-11
6   2019-11-11
7   2019-11-13
8   2019-11-13
9   2019-11-15
Name: 入院时间1, dtype: datetime64[ns]

In [17]:
target['入院时间1']=pd.to_datetime(target['入院时间'],format='%Y.%m.%d',errors='coerce')
target['出院时间1']=pd.to_datetime(target['出院时间'],format='%Y.%m.%d',errors='coerce')

In [18]:
target['入院时间1']

0     2019-10-28
1     2019-11-01
2     2019-11-04
3     2019-11-04
4     2019-11-08
         ...    
572   2018-06-22
573   2018-08-31
574   2018-12-18
575   2019-01-23
576   2018-09-21
Name: 入院时间1, Length: 577, dtype: datetime64[ns]

In [28]:
target[target['入院时间1'].dt.year==2018].to_excel("./2018.xlsx",sheet_name="Sheet1",index=False)

In [29]:
target[target['入院时间1'].dt.year==2019].to_excel("./2019.xlsx",sheet_name="Sheet1",index=False)

In [30]:
target[target['入院时间1'].dt.year==2020].to_excel("./2020.xlsx",sheet_name="Sheet1",index=False)

In [ ]:
import process,time
result = pd.DataFrame(columns = fd.columns) # 错误结果 
matched =  pd.DataFrame(columns = fd.columns)  # 正确结果
drop_index=[] #记录需要删除对行号

max_steps=target.shape[0]
process_bar = process.ShowProcess(max_steps, 'OK')
for idx,row in target.iterrows():
    # print(type(row))
    # print(row["住院号"])
    fil=(fd["住院号"]==row["住院号"]) \
        &(fd["科室"]==row["科室"]) \
        &(fd["姓名"]==row["姓名"]) \
        &(fd["借支金额"]==row["借支金额"]) \
        &(fd["还款金额"]==row["还款金额"]) \
        &(fd["还款金额.1"]==row["还款金额.1"]) \
        &(fd["还款金额.2"]==row["还款金额.2"]) 
    _temp= fd[fil]  
    if len(_temp)==0: # can not match
        # print("{} is false".format(idx))
        result.loc[len(result)] = row
    else:
        # print("{} is true,fd index is {}".format(idx,_temp.index.tolist()))
        matched.loc[len(matched)] = row
        drop_index.extend(_temp.index.tolist())
    process_bar.show_process()
    time.sleep(0.01)
# print(result)
# write to error
result.to_excel("./result.xlsx",sheet_name="Sheet1",index=False)
matched.to_excel("./matched.xlsx",sheet_name="Sheet1",index=False)
# print(drop_index)
left=fd.drop(drop_index) # 剩余对比结果
left.to_excel("./left.xlsx",sheet_name="Sheet1",index=False)
print("对比结果如下：")
print("-----------------------------------")
print("一共对比记录："+str(target.shape[0]))
print("其中，正确比对合计："+str(matched.shape[0]))
print("错误对比结果合计："+str(result.shape[0]))
print("原始结果合计："+str(fd.shape[0]))
print("剩余对比结果合计："+str(left.shape[0]))
print("-----------------------------------")
print("最后产生三个文件：")
print("错误结果：result.xlsx")
print("正确结果：matched.xlsx")
print("剩余结果：left.xlsx")
print("-----------------------------------")

# left.shape

### 例子： convert data to datetime

In [ ]:
import pandas as pd

df=pd.read_excel("./base.xls",sheet_name="Sheet1")
# df.head(10)
# fd.dtypes
# fd.set_index(pd.to_datetime(fd['借支日期']),inplace=True)
# fd['借支日期']=(pd.to_datetime(fd['借支日期'],format='%Y%m%d'))
# fd['xi借支日期'] = fd['借支日期'].astype(str)
df['还款日期'] = pd.to_datetime(df['还款日期'].astype(str), format='%Y%m%d.0', errors='coerce')
df['借支日期'] = pd.to_datetime(df['借支日期'].astype(str), format='%Y%m%d.0', errors='coerce')
df
# fd['借支日期'] = fd['借支日期'].astype('datetime64[ns]')
# fd.head(20)

In [ ]:
import pandas as pd
pd.to_datetime(pd.Series([20100329.0, 20100328.0, 20100329.0]).astype(str), format='%Y%m%d.0')

In [ ]:
import numpy as np
target=pd.read_excel("./base.xls",sheet_name="Sheet1")
target.fillna(0, inplace=True)
target.replace(0,np.nan,inplace=True)
target.head(10)
target.to_excel("./test.xlsx",sheet_name="Sheet1",index=False)